In [0]:
spark.sql("""
CREATE EXTERNAL LOCATION IF NOT EXISTS adls_bronze URL 'abfss://bronze@databdls.dfs.core.windows.net/'
    WITH (STORAGE CREDENTIAL `formula1-adls-credential`)
    COMMENT 'Default source for Azure exernal data';
""")

In [0]:
spark.sql("""
CREATE CATALOG IF NOT EXISTS dev_bronze_f1_catalog
managed LOCATION 'abfss://bronze@databdls.dfs.core.windows.net'
""")
spark.sql("""
CREATE SCHEMA IF NOT EXISTS dev_bronze_f1_catalog.dev_bronze_f1_schema
""")

In [0]:
spark.sql("USE CATALOG dev_bronze_f1_catalog")
spark.sql("USE SCHEMA dev_bronze_f1_schema")

In [0]:
# delete everything from volume
# dbutils.fs.rm("/Volumes/dev_bronze_f1_catalog/dev_bronze_f1_schema/target_volume_for_copy", recurse=True)

In [0]:
spark.sql("""
CREATE VOLUME IF NOT EXISTS target_volume_for_copy
""")

In [0]:
# create folder in volume
dbutils.fs.mkdirs("/Volumes/dev_bronze_f1_catalog/dev_bronze_f1_schema/target_volume_for_copy/input")

In [0]:
df = (spark.read
  .format("csv")
  .option("header", True)
  .option("inferSchema", True)
  .load("abfss://raw@databdls.dfs.core.windows.net/2021-03-21/races.csv")
)
df.write.mode("overwrite").format("csv").option("header", True).save("/Volumes/dev_bronze_f1_catalog/dev_bronze_f1_schema/target_volume_for_copy/input/1.csv")

In [0]:
# spark.sql("DROP CATALOG IF EXISTS dev_bronze_f1_catalog CASCADE")
# spark.sql("DROP EXTERNAL LOCATION abfss_remote")
spark.sql("DROP VOLUME IF EXISTS target_volume_for_copy")

In [0]:
df = spark.read.format("csv").option("header", True).load("/Volumes/dev_bronze_f1_catalog/dev_bronze_f1_schema/target_volume_for_copy/input/1.csv")
display(df)

In [0]:
# recovered deleted managed volume by history 
display(spark.sql("""
DESCRIBE HISTORY target_volume_for_copy
"""))

In [0]:
display(spark.sql("""
SELECT * FROM csv.`/Volumes/dev_bronze_f1_catalog/dev_bronze_f1_schema/target_volume_for_copy/input/1.csv`
"""))

In [0]:
%sql
-- creating placeholder table
create table if not exists placeholder_vol

In [0]:
spark.sql("""
COPY INTO placeholder_vol
FROM '/Volumes/dev_bronze_f1_catalog/dev_bronze_f1_schema/target_volume_for_copy/input/1.csv'
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true')
""")

In [0]:
%sql
select count(*) from placeholder_vol limit 1

In [0]:
%sh
ls -ltr /

In [0]:
dbutils.fs.ls('/')

## ### EXTERNAL volume

In [0]:
spark.sql("""
CREATE EXTERNAL LOCATION IF NOT EXISTS adls_raw URL 'abfss://raw@databdls.dfs.core.windows.net/'
    WITH (STORAGE CREDENTIAL `formula1-adls-credential`)
    COMMENT 'Default source for Azure exernal data';
""")

In [0]:
# Did not allow to create external volume at raw container because it overlaps with managed storage location i dont know why so i created sub folder
spark.sql("""
CREATE external VOLUME IF NOT EXISTS my_external_volume
LOCATION 'abfss://raw@databdls.dfs.core.windows.net/ext_vol'
""")

In [0]:
spark.sql("DROP VOLUME IF EXISTS my_external_volume")

In [0]:
import pandas as pd

data = {'id': [1, 2, 3], 'name': ['Alice', 'Bob', 'Charlie']}
pdf = pd.DataFrame(data)
df = spark.createDataFrame(pdf)
df.write.mode("overwrite").option("header", True).csv("/Volumes/dev_bronze_f1_catalog/dev_bronze_f1_schema/my_external_volume/sample.csv")

In [0]:
%sql
select * from csv.`/Volumes/dev_bronze_f1_catalog/dev_bronze_f1_schema/my_external_volume/sample.csv`